In [1]:
!pip install tqdm
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 5.6 MB/s eta 0:00:0000:0100:01


In [27]:
# Declare import statements
import nltk
import xml.etree.ElementTree as ET
from xml.dom import minidom
import pandas as pd
pd.options.mode.chained_assignment = None  
from tqdm import tqdm
# Download nltk addons
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/Trinity_1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Trinity_1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [37]:
file_loc = '/Users/Trinity_1/Research/EFCAMDAT Subcorpus (Clean)/' ## CHANGE THIS TO THE FILE CONTAINING THE XLM FILE
xml_file_name = 'Arabic.xml'
xml_file = minidom.parse('{}{}'.format(file_loc, xml_file_name))

In [38]:
# Parse the xml file into pandas dataframe

grades = xml_file.getElementsByTagName('grade')
writing = xml_file.getElementsByTagName('writing')
text_xml = xml_file.getElementsByTagName('text')
learner = xml_file.getElementsByTagName('learner')
topic = xml_file.getElementsByTagName('topic')

grade = [] 
text = []
ids = []
writing_ids = []
levels = []
unit = []
nationality = []
topic_id = []
topic_text = []

for ind in range(len(grades)):
    grade.append(float(grades[ind].firstChild.nodeValue))
    text.append((text_xml[ind].firstChild.nodeValue))
    
    ids.append(int(learner[ind].attributes['id'].value))
    nationality.append(str(learner[ind].attributes['nationality'].value))
    
    writing_ids.append(int(writing[ind].attributes['id'].value))
    levels.append(int(writing[ind].attributes['level'].value))
    unit.append(int(writing[ind].attributes['unit'].value))
    
    topic_id.append(int(topic[ind].attributes['id'].value))
    topic_text.append(str(topic[ind].firstChild.nodeValue))

ensemble = {
  "learner_id": ids,
  "grades": grade,
  "nationality": nationality,
  "writing_id": writing_ids,
  "levels": levels,
  "unit":unit,
  "topic_id": topic_id,
  "topic_text":topic_text,
  "passage": text,}

corpus = pd.DataFrame(ensemble)

In [39]:
# Look at the first 5 entries in the corpus:
corpus.head()

,learner_id,grades,nationality,writing_id,levels,unit,topic_id,topic_text,passage
0,1801,80.0,kw,153,1,2,2,Taking inventory in the office,"\n Dear Ms There are 30 pens , 15 pens , ..."
1,1801,89.0,kw,154,1,3,3,Writing an online profile,\n Hi ! My name's Anas I'm 25 years old ....
2,43077,80.0,sa,210,4,1,25,Writing about what you do,\n My name is Saad . l am 21 years old . ...
3,166681,90.0,sa,281,4,1,25,Writing about what you do,"\n I'm working housewife, cleaner home, c..."
4,166681,73.0,sa,282,4,2,26,Describing routines,"\n Hello, I'm Malak , I'm working housew..."


In [40]:
#Aux function to add a sentence to the dataframe

def add_query_found(corpus, sentence):
    sliced = corpus
    sliced["query_sentence"] = sentence
    return sliced

In [41]:
df_entry = []

for learner in tqdm(range(len(corpus))):
    
    # Tokenize the passage and convert the tokenize passage into 
    # tags.
    
    tokenize_paragraph = nltk.word_tokenize(corpus['passage'][learner])
    tagged_paragraph = nltk.pos_tag(tokenize_paragraph)
    
    # Using the '.' grammar tag to parse passages into sentences.
    # This returns the position of the end points in the tagged paragraphs.
    
    endline_position = [i + 1 for i, v in enumerate(tagged_paragraph) if v[1] == '.']
    
    # For each identified sentence, we parse out one sentence.
    
    for index in range(len(endline_position)):
        if index == 0:
            tagged_sentence = tagged_paragraph[:endline_position[index]]
            sentence = tokenize_paragraph[:endline_position[index]]
        else:
            tagged_sentence = tagged_paragraph[endline_position[index-1]:endline_position[index]]
            sentence = tokenize_paragraph[endline_position[index-1]:endline_position[index]]
        
        
        ###==================[PARSE SENTENCE STRUCTURE HERE]====================
        # Can be changed to tagged_sentence[0][1] = 'VBZ'
        # tagged_sentence[-1][0] != '?' removed sentences ending with question
        # 
        if tagged_sentence[0][1][0:3] =='VBZ' and tagged_sentence[-1][0] != '?':
            df_entry.append(add_query_found(corpus.loc[learner], ' '.join(sentence)))
            
query_df = pd.DataFrame(df_entry)         

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60383/60383 [01:35<00:00, 631.61it/s]


In [43]:
query_df = pd.DataFrame(df_entry)
query_df.reset_index(drop=True, inplace=True)

In [44]:
pd.set_option('display.max_rows', None)
query_df["query_sentence"]

0                              Lives in : Saudi Aribia .
1             Is she beautiful.2- This is my boyfriend .
2                       Lets eat cake , dance and sing .
3      Sometimes it 's tiring and boring when I work ...
4                      IN THE EVENING , I meet friends .
5                              Lives in : saudi Arabia .
6                       Lets eat cake , dance and sing .
7      Lets eat cake , dance , play and sing.The part...
8                        Sets the table in the morning .
9                             Does ironing on Thursday .
10     has mother is a frindly women and has father i...
11                      Lets eat cake , dance and sing .
12                      Lets eat cake , dance and sing .
13                        Plays computer games at 6 pm .
14                       Does gardening in the morning .
15                        Sets the table at 6 p m. You .
16                                           lets roll .
17                      Lets ea

In [45]:
# Save the found sentences out as a csv.
# add query_df[["learner_id","query_sentence"]] to output a subset to csv.

query_df.to_csv('{}/{}.csv'.format(file_loc,xml_file_name[:-4]))